# Connexion a la bdd

In [ ]:
!pip install langchain
!pip install neo4j
!pip install openai
!pip install tiktoken

In [ ]:
from openai import OpenAI
from langchain.graphs import Neo4jGraph
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from neo4j import GraphDatabase


In [ ]:
os.environ['OPENAI_API_KEY'] = ""

client = OpenAI()

url = ""  # Replace with your Neo4j server URI
username = ""
password = ""

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Content",
    text_node_properties=['contenu'],
    embedding_node_property='embedding',
)

# Test sur films

## Our method

In [ ]:
def run_cypher_query(query):
    with GraphDatabase.driver(url, auth=(username, password)) as driver:
        with driver.session() as session:
            result = session.run(query)
            records = list(result)
            result.consume()
            return records

def voisin_reponses(question,vector_index):
  response = vector_index.similarity_search(question)
  reponses = []
  for res in response:
      texte_recherche = res.page_content.split("contenu: ")[1]
      reponses.append(texte_recherche)
      cypher_query = f"MATCH (c:Content)-[:has_commun_entity]->(e:Content) WHERE c.contenu = '{texte_recherche}' RETURN DISTINCT e.contenu as contenu;"
      resultats = run_cypher_query(cypher_query)
      for record in resultats[:10]:
          reponses.append(record["contenu"])
  return list(set(reponses))

def reponse_question(question,vector_index, client):
  facts = voisin_reponses(question,vector_index)
  assistant = "here's a set of facts: " + str(facts)
  prompt_template = """
    give me an answer to this question based only on the set of facts: {}.
    Some fo these facts may not contain information necessery to the answer, you can ignore them.
    Dont add information from your knowledge, if you dont have the information in the set of facts dont add answer from your own.
    """.format(question)

  messages = [
              {"role": "user", "content": prompt_template},
              {"role": "assistant", "content": assistant},
              ]

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  return response.choices[0].message.content

In [ ]:
question = "tell me about spider man, and the cast of the film far from home."
print(reponse_question(question, vector_index, client))

Spider-Man, played by Tom Holland, is a high school junior and Avenger who received spider-like abilities after being bitten by a radioactive spider. In the film "Far From Home," he is joined by a cast that includes Samuel L. Jackson as Nick Fury, Zendaya as Michelle MJ Jones-Watson, Cobie Smulders as Maria Hill, Jon Favreau as Happy Hogan, J.B. Smoove as Julius Dell, Jacob Batalon as Ned Leeds, Martin Starr as Roger Harrington, Tony Revolori as Eugene Flash Thompson, and Marisa Tomei as May Parker. Jake Gyllenhaal plays Quentin Beck, also known as Mysterio, a former Stark Industries employee and holographic/illusions specialist who masquerades as a superhero from Earth-833 in the multiverse. Other cast members include Angourie Rice, Jorge Lendeborg Jr, Numan Acar, Remy Hii, Claire Rushbrook, Zach Barack, and Zoha Rahman. The film also features cameo appearances by actors such as J.K. Simmons as J. Jonah Jameson and appearances by Skrulls Talos and Soren, played by Ben Mendelsohn and S

In [ ]:
question = "what are the harry potter movies you know?"
print(reponse_question(question, vector_index, client))

The Harry Potter movies I know are Harry Potter and the Philosophers Stone, Harry Potter and the Chamber of Secrets, Harry Potter and the Goblet of Fire, Harry Potter and the Order of the Phoenix, Harry Potter and the Half-Blood Prince, and Harry Potter and the Deathly Hallows Part 1 and Part 2.


In [ ]:
question = "what movies did leonardo dicaprio played?"
print(reponse_question(question, vector_index, client))

Leonardo DiCaprio played the role of Jay Gatsby in the 2013 film "The Great Gatsby".


In [ ]:
question = "give me the movies that have the same production company"
print(reponse_question(question, vector_index, client))

Interstellar and Dune Part Two have the same production company, Warner Bros.


In [ ]:
question = "give me a resume about uncharted the film. also tell me more about inception."
print(reponse_question(question, vector_index, client))

Uncharted is a 2022 American action adventure film directed by Ruben Fleischer and based on the video game franchise of the same name. It stars Tom Holland as Nathan Drake and Mark Wahlberg as Victor Sullivan. The film follows Drake and Sullivan in a race against corrupt billionaire Santiago Moncada and mercenary leader Jo Braddock to locate the fabled treasure of the Magellan expedition. The film received mixed reviews from critics, with some praising the visual effects and cast performances, while others found it lacking in character development and originality.

Inception is a 2010 science fiction film directed by Christopher Nolan. The film follows a thief who enters the dreams of others to steal their secrets. Inception received critical acclaim for its visual effects, performances, and originality. The film explores themes of reality and dreams, and has been praised for its complex narrative structure.


In [ ]:
question = "give me the set of persons who wrote and produced the same movie."
print(reponse_question(question, vector_index, client))

The set of persons who wrote and produced the same movie based on the provided set of facts are:

- Chris Columbus
- Steve Kloves
- David Heyman
- Denis Villeneuve
- Joe Walker
- Brad Riker
- Patrice Vermette
- Paul Lambert
- Gerd Nefzer
- Thomas Struthers
- Tanya Lapointe
- Brian Herbert
- Byron Merritt
- Kim Herbert
- Thomas Tull
- Jon Spaihts
- Richard P. Rubinstein
- John Harrison
- Herbert W. Gain
- Kevin J. Anderson


In [ ]:
question = "What are the movies in which Zendaya has appeared?"
print(reponse_question(question, vector_index, client))

The movies in which Zendaya has appeared are Spider-Man: Homecoming (2017), Spider-Man: Far From Home (2019), and Spider-Man: No Way Home (2021).


In [ ]:
question = "brief description of all movies where decaprio played. also give me the name of all movies where zendaya acted."
print(reponse_question(question, vector_index, client))

Leonardo DiCaprio has played in the following movies: Inception

Zendaya has acted in the following movies: Spider-Man: Far From Home


In [ ]:
question = "who are the actors of hunger games. Also give some information about oppenheimer and interstellar and dune and the greate gatsby and Harry Potter and the Deathly Hallows Part 1."
print(reponse_question(question, vector_index, client))

The actors of Hunger Games are Jennifer Lawrence, Josh Hutcherson, and Liam Hemsworth. 

Oppenheimer is a 2023 epic biographical thriller film written, directed, and co-produced by Christopher Nolan, starring Cillian Murphy as J. Robert Oppenheimer. The film also features Emily Blunt, Matt Damon, Robert Downey Jr., and Florence Pugh among others. It explores the life of Oppenheimer, his work on the Manhattan Project, and his eventual fall from grace.

Interstellar is a 2014 epic science fiction film directed by Christopher Nolan, starring Matthew McConaughey, Anne Hathaway, Jessica Chastain, and Michael Caine. The film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humanity.

Dune is a 2021 epic science fiction film directed by Denis Villeneuve, based on the 1965 novel by Frank Herbert. The film stars Timothee Chalamet, Zendaya, Rebecca Ferguson, and Josh Brolin. It follows Paul Atreides as he unites with the Fremen people of the des